In [ ]:
# ===== Test-all-steps from saved artifacts =====
from pathlib import Path
import torch, random, json
from itertools import product
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
)

# ---------- 1) بارگذاری مدل و توکنایزر Fine-Tuned ----------
SAVE_PATH = Path("ft_gpt2_chat")           # پوشه‌ای که در سلول قبل ساخته شد
device     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_ft = AutoTokenizer.from_pretrained(SAVE_PATH)
model_ft     = AutoModelForCausalLM.from_pretrained(SAVE_PATH).to(device)

# ---------- 2) تعریف پرامپت‌ها و شبکهٔ پارامترها ----------
prompts = {
    "question":       "Why do cats seem to prefer sleeping on laptops?",
    "declarative":    "The future of renewable energy lies in",
    "incomplete":     "On a bright summer morning,",
    "narrative_seed": "She opened the dusty book and discovered",
    "code_comment":   "# TODO: Refactor the user authentication logic so that",
}

grid = list(product([0.7, 1.0], [50, 100], [0.9, 0.95]))  # (temperature, top_k, top_p)
random.shuffle(grid)                                      # کمی تصادفی

# ---------- 3) تولید نمونه‌ها با مدل Fine-Tuned ----------
print("🔹 Generations – fine-tuned GPT-2\n")
records_ft = []
for name, prompt in prompts.items():
    for temperature, top_k, top_p in grid[:6]:            # فقط ۶ ترکیب اول برای اختصار
        inputs = tokenizer_ft(prompt, return_tensors="pt").to(device)
        out_ids = model_ft.generate(
            **inputs,
            max_new_tokens=60,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer_ft.eos_token_id,
        )
        txt = tokenizer_ft.decode(out_ids[0], skip_special_tokens=True)
        print(f"---\\n{prompt}  (T={temperature}, k={top_k}, p={top_p})\\n{txt}\\n")
        records_ft.append(
            dict(model="ft_gpt2_chat", prompt=prompt,
                 temperature=temperature, top_k=top_k, top_p=top_p, output=txt)
        )

# ---------- 4) مقایسهٔ سریع با مدل پایهٔ gpt2 برای سه پرامپت ----------
print("\\n🔹 Quick size / domain comparison (base gpt2 vs fine-tuned)\\n")
base_tokenizer = AutoTokenizer.from_pretrained("gpt2")
base_model     = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

sample_prompts = [
    "Explain quantum entanglement like I'm five:",
    "In the year 2075, humans finally",
    "Svelte is a frontend framework that",
]

gen_args = dict(max_new_tokens=80, temperature=0.9, top_k=50, top_p=0.92, do_sample=True)

rows = []
for p in sample_prompts:
    # base model
    base_ids = base_tokenizer(p, return_tensors="pt").to(device)
    base_out = base_model.generate(**base_ids, **gen_args,
                                   pad_token_id=base_tokenizer.eos_token_id)
    base_txt = base_tokenizer.decode(base_out[0], skip_special_tokens=True)

    # fine-tuned model
    ft_ids = tokenizer_ft(p, return_tensors="pt").to(device)
    ft_out = model_ft.generate(**ft_ids, **gen_args,
                               pad_token_id=tokenizer_ft.eos_token_id)
    ft_txt = tokenizer_ft.decode(ft_out[0], skip_special_tokens=True)

    rows.append(dict(prompt=p, gpt2_base=base_txt, gpt2_ft=ft_txt))
    print(f"=== Prompt: {p}\\n[base] {base_txt}\\n[ft  ] {ft_txt}\\n")

# ---------- 5) ذخیرهٔ خروجی‌ها برای بررسی آفلاین ----------
Path("eval_outputs").mkdir(exist_ok=True)
with open("eval_outputs/task2_generations_ft.json", "w") as f:
    json.dump(records_ft, f, indent=2)
pd.DataFrame(rows).to_csv("eval_outputs/size_comparison.csv", index=False)

print("\\n✅ All tests finished – results saved in eval_outputs/")
